In [188]:
import pandas as pd
from ipywidgets import IntProgress
from IPython.display import display
import statsmodels.api as sm
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
from sklearn.preprocessing import normalize
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
import scipy.stats as stats
from IPython.display import HTML, Math
from decimal import Decimal

import warnings
warnings.filterwarnings('ignore')

In [189]:
tracks = pd.read_csv('fma_metadata/tracks.csv', header=[0,1,2])
tracks.head(100)

Unnamed: 0_level_0              album                       \
   Unnamed: 0_level_1           comments         date_created   
             track_id Unnamed: 1_level_2   Unnamed: 2_level_2   
0                   2                  0  2008-11-26 01:44:45   
1                   3                  0  2008-11-26 01:44:45   
2                   5                  0  2008-11-26 01:44:45   
3                  10                  0  2008-11-26 01:45:08   
4                  20                  0  2008-11-26 01:45:05   
5                  26                  0  2008-11-26 01:45:05   
6                  30                  0  2008-11-26 01:45:05   
7                  46                  0  2008-11-26 01:45:05   
8                  48                  0  2008-11-26 01:45:05   
9                 134                  0  2008-11-26 01:44:45   
10                135                  1  2008-11-26 01:49:19   
11                136                  1  2008-11-26 01:49:19   
12                137                  1  2008-11-26 01:49:35   
13                138                  1  2008-11-26 01:49:35   
14                139                  0  2008-11-26 01:49:57   
15                140                  1  2008-11-26 01:49:59   
16                141                  0  2008-11-26 01:49:57   
17                142                  0  2008-11-26 01:50:03   
18                144                  0  2008-11-26 01:50:07   
19                145                  0  2008-11-26 01:50:07   
20                146                  0  2008-11-26 01:50:13   
21                147                  0  2008-11-26 01:50:13   
22                148                  0  2008-11-26 01:50:19   
23                149                  0  2008-11-26 01:50:25   
24                150                  0  2008-11-26 01:50:25   
25                151                  0  2008-11-26 01:50:46   
26                152                  0  2008-11-26 01:50:46   
27                153                  0  2008-11-26 01:50:50   
28                154                  0  2008-11-26 01:50:50   
29                155                  0  2008-11-26 01:51:00   
..                ...                ...                  ...   
70                199                  0  2008-11-26 01:52:33   
71                200                  0  2008-11-26 01:52:31   
72                201                  0  2008-11-26 01:52:33   
73                202                  0  2008-11-26 01:52:33   
74                203                  0  2008-11-26 01:52:31   
75                204                  0  2008-11-26 01:52:31   
76                205                  0  2008-11-26 01:52:33   
77                206                  0  2008-11-26 01:52:33   
78                207                  0  2008-11-26 01:52:31   
79                208                  0  2008-11-26 01:52:31   
80                209                  0  2008-11-26 01:52:31   
81                210                  0  2008-11-26 01:52:31   
82                211                  0  2008-11-26 01:52:31   
83                212                  0  2008-11-26 01:52:31   
84                213                  0  2008-11-26 01:53:47   
85                226                  0  2008-11-26 01:54:25   
86                228                  0  2008-11-26 01:54:25   
87                236                  0  2008-11-26 01:54:55   
88                237                  0  2008-11-26 01:55:00   
89                238                  0  2008-11-26 01:55:00   
90                246                  0  2008-11-26 01:55:43   
91                247                  0  2008-11-26 01:55:44   
92                248                  0  2008-11-26 01:55:47   
93                249                  0  2008-11-26 01:55:44   
94                250                  0  2008-11-26 01:55:47   
95                251                  0  2008-11-26 01:55:47   
96                252                  0  2008-11-26 01:55:44   
97                253                 

In [190]:
for v in tracks:
    print("{}:{}".format(v[0],v[1]))
for v in tracks:
    print(v)

Unnamed: 0_level_0:Unnamed: 0_level_1
album:comments
album:date_created
album:date_released
album:engineer
album:favorites
album:id
album:information
album:listens
album:producer
album:tags
album:title
album:tracks
album:type
artist:active_year_begin
artist:active_year_end
artist:associated_labels
artist:bio
artist:comments
artist:date_created
artist:favorites
artist:id
artist:latitude
artist:location
artist:longitude
artist:members
artist:name
artist:related_projects
artist:tags
artist:website
artist:wikipedia_page
set:split
set:subset
track:bit_rate
track:comments
track:composer
track:date_created
track:date_recorded
track:duration
track:favorites
track:genre_top
track:genres
track:genres_all
track:information
track:interest
track:language_code
track:license
track:listens
track:lyricist
track:number
track:publisher
track:tags
track:title
('Unnamed: 0_level_0', 'Unnamed: 0_level_1', 'track_id')
('album', 'comments', 'Unnamed: 1_level_2')
('album', 'date_created', 'Unnamed: 2_level_2')

In [191]:
features = [('album', 'comments', 'Unnamed: 1_level_2'),
('album', 'date_created', 'Unnamed: 2_level_2'),
('album', 'date_released', 'Unnamed: 3_level_2'),
('album', 'favorites', 'Unnamed: 5_level_2'),
('album', 'listens', 'Unnamed: 8_level_2'),
('album', 'tags', 'Unnamed: 10_level_2'),
('album', 'tracks', 'Unnamed: 12_level_2'),
('album', 'type', 'Unnamed: 13_level_2'),
('artist', 'active_year_begin', 'Unnamed: 14_level_2'),
('artist', 'active_year_end', 'Unnamed: 15_level_2'),
('artist', 'comments', 'Unnamed: 18_level_2'),
('artist', 'date_created', 'Unnamed: 19_level_2'),
('artist', 'favorites', 'Unnamed: 20_level_2'),
('artist', 'latitude', 'Unnamed: 22_level_2'),
('artist', 'longitude', 'Unnamed: 24_level_2'),
('artist', 'tags', 'Unnamed: 28_level_2'),
('set', 'split', 'Unnamed: 31_level_2'),
('set', 'subset', 'Unnamed: 32_level_2'),
('track', 'comments', 'Unnamed: 34_level_2'),
('track', 'date_created', 'Unnamed: 36_level_2'),
('track', 'date_recorded', 'Unnamed: 37_level_2'),
('track', 'duration', 'Unnamed: 38_level_2'),
('track', 'favorites', 'Unnamed: 39_level_2'),
('track', 'genre_top', 'Unnamed: 40_level_2'),
('track', 'genres', 'Unnamed: 41_level_2'),
('track', 'genres_all', 'Unnamed: 42_level_2'),
('track', 'interest', 'Unnamed: 44_level_2'),
('track', 'language_code', 'Unnamed: 45_level_2'),
('track', 'listens', 'Unnamed: 47_level_2'),
('track', 'number', 'Unnamed: 49_level_2'),
('track', 'tags', 'Unnamed: 51_level_2')]

df = pd.DataFrame()
for v in features:
    df["{}:{}".format(v[0],v[1])] = tracks[v]

Y_col = 'track:genre_top'
date_l = ['album:date_created', 'album:date_released', 'artist:active_year_begin', 'artist:active_year_end', 'artist:date_created', 'track:date_created', 'track:date_recorded']
one_hot_l = ['album:type', 'track:language_code']
one_hot_c = ['album:tags', 'artist:tags', 'track:genres', 'track:genres_all', 'track:tags']
exclude_l = ['set:split', 'set:subset']

for d in date_l:
    df[d] = pd.to_numeric(pd.to_datetime(df[d]))

Y_cols = []
dummy = pd.get_dummies(df[Y_col])
for v in dummy:
    df["{}:{}".format(Y_col, v)] = dummy[v]
    Y_cols.append("{}:{}".format(Y_col, v))

for c in df:
    if df[c].dtype in ["float64", "int64"] and df[c].isna().sum() > 0:
        df[c] = df[c].fillna(df[c].mean())

for o in one_hot_l:
    dummy = pd.get_dummies(df[o])
    for v in dummy:
        df["{}:{}".format(o, v)] = dummy[v]
    df = df.drop(columns=[o])

# df = df.dropna(subset=[Y_col])
y_cats = list(df[Y_col].astype('category').cat.categories)
df[Y_col] = df[Y_col].astype('category').cat.codes
if sorted(list(set(list(df[Y_col]))))[0] == -1:
    y_cats = ["None"] + y_cats
n=1000
weights = {}
cvs = []
cv_num = 3
while True:
    cvs = []
    cont = True
    df_samp = df.sample(n=cv_num*n)
    df_rest = df_samp.copy()
    for i in range(cv_num):
        df_sub = df_rest.sample(n=n)
        cvs.append(list(df_sub.index))
        df_rest = df_rest.loc[~df_rest.index.isin(df_sub.index)]
    weights = {}
    cv_l = []
    for i in range(cv_num):
        cv_l.append(sorted(list(set(list(df_samp[Y_col][cvs[i]])))))
    for i in sorted(list(set(list(df[Y_col])))):
        weights[i] = 1.0/(df[Y_col] == i).sum()
        for l in cv_l:
            if i not in l:
                cont = False
                break
    m_w = weights[min(weights, key=weights.get)]
    if cont:
        for i in weights:
            weights[i] = int(weights[i] / m_w)
        break
split = []
for v in cvs:
    split.append(([j for i in cvs for j in i if i != v], v))

In [192]:
cols = [x for x in df_samp if x not in one_hot_l+one_hot_c+exclude_l+Y_cols+[Y_col]]
X = df_samp[cols]
Y = df_samp[Y_col]

print(X.shape)

model = LogisticRegression(multi_class='multinomial', solver='sag', max_iter=1000, class_weight=weights)
rfe = RFECV(model, cv=3, step=4, verbose=1, n_jobs=-1)
rfe = rfe.fit(X, Y)
Y_pred = rfe.predict(df[cols])
print(rfe.score(df[cols],df[Y_col]))
print(sorted(list(set(list(Y_pred)))))

(3000, 70)
Fitting estimator with 70 features.
Fitting estimator with 66 features.
Fitting estimator with 62 features.
Fitting estimator with 58 features.
Fitting estimator with 54 features.
Fitting estimator with 50 features.
Fitting estimator with 46 features.
Fitting estimator with 42 features.
Fitting estimator with 38 features.
Fitting estimator with 34 features.
Fitting estimator with 30 features.
0.013558654080732636
[0, 1, 2, 4, 5, 6, 8, 10, 11, 12, 13, 14]


In [193]:
display(HTML("<script src='https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.5/MathJax.js?config=TeX-MML-AM_CHTML'></script>"))
out = r'$$\begin{eqnarray} P(k(c)) & = & \frac{\exp(k(c))}{1 + \exp(k(c))} \\\\\\'
pred_cols = [x for i,x in enumerate(cols) if rfe.support_[i]]
for i in range(len(y_cats)):
    s = "{:.8E}".format(float(rfe.estimator_.intercept_[i]))
    out += r'k({}) & = & {}{:.8E} \\'.format(y_cats[i], r"\;\:" * (16 - len(s)), Decimal(float(rfe.estimator_.intercept_[i])))
    for j in range(len(pred_cols)):
        coef = float(rfe.estimator_.coef_[i][j])
        if coef < 0:
            s = "{:.8E}".format(Decimal(-coef))
            out += " & - & {}{:.8E} ".format(r"\;\:" * (16 - len(s)), Decimal(-coef))+r" \cdot "+r"\_".join(pred_cols[j].split("_"))+r"\\"
        else:
            s = "{:.8E}".format(Decimal(coef))
            out += " & + & {}{:.8E} ".format(r"\;\:" * (16 - len(s)), Decimal(coef))+r" \cdot "+r"\_".join(pred_cols[j].split("_"))+r"\\"
    out += "\\\\ ACC({}) & = & {: 12.8f} \\\\\\\\\\\\".format(y_cats[i], float(rfe.score(df[df[Y_col] == i-1][cols], df[df[Y_col] == i-1][Y_col])))         
out += r"\end{eqnarray}$$"
display(Math(r'{}'.format(out)))

<IPython.core.display.Math object>

In [194]:
display(HTML("<script src='https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.5/MathJax.js?config=TeX-MML-AM_CHTML'></script>"))
out = r'$$'
f_rank = []
for ind,i in enumerate(rfe.ranking_):
    f_rank.append((i, cols[ind]))
curr = -1
for f in sorted(f_rank):
    if f[0] != curr:
        index = "{}".format(f[0])
        index = (4-len(index))*r"\;\:" + index
        out += '{}\;\:{}\\\\'.format(index, r"\_".join(f[1].split("_")))
        curr = f[0]
    else:
        out += '\;\:\;\:\;\:\;\:\;\:{}\\\\'.format(r"\_".join(f[1].split("_")))
out += '$$'
display(Math(r'{}'.format(out)))

<IPython.core.display.Math object>